In [17]:
import numpy as np
import os
from pycbc.io import HFile, get_all_subkeys
from pycbc.events import coinc
import pycbc.io

In [2]:
def get_trigger_info(ttype, sfile, bank, merge_trig_temp):
    '''Read all relevant information
    '''
    merge_fields = ['coa_phase', 'snr']
    ifos = sfile.attrs['ifos'].split(' ') #ifos toma el atributo ifos del archivo sfile y lo divide en una lista de interferómetros, que se usan para acceder a datos específicos de cada detector
    
    params = {}
    params['stat'] = sfile[f'{ttype}/stat'.format(section)][:]
    if 'ifar' in sfile[f'{ttype}'.format(section)]:
        params['ifar'] = sfile[f'{ttype}/ifar'.format(section)][:]
    elif 'ifar_exc' in sfile[f'{ttype}'.format(section)]:
        params['ifar'] = sfile[f'{ttype}/ifar_exc'.format(section)][:]
    else:
        raise KeyError("ifar or ifar_exc does not found in")
        
    bank_ids = sfile[f'{ttype}/template_id'.format(section)][:]
    
    for ifo in ifos:
        params[f'time_{ifo}'] = sfile[f'{ttype}/{ifo}/time'][:]
        params[f'tid_{ifo}'] = sfile[f'{ttype}/{ifo}/trigger_id'][:]
        tids = sfile[f'{ttype}/{ifo}/trigger_id'][:]
        tids = tids[tids >= 0]
        # print(ifo, tids.shape, params[f'tid_{ifo}'].shape)
        
        sngls = pycbc.io.SingleDetTriggers(merge_trig_temp.format(ifo), bank_file, 
                                None, None, None, detector=ifo, premask=tids)
        for field in merge_fields:
            print(f"reading field {field} for {ifo}")
            params[f'{field}_{ifo}'] = -1 * np.ones_like(params[f'time_{ifo}'])
            # print(field, f'{field}_{ifo}', params[f'{field}_{ifo}'].shape)
            # These are idexings and hence RAM heavy
            try:
                params[f'{field}_{ifo}'][params[f'tid_{ifo}'] >= 0] = getattr(sngls, f'{field}')
            except AttributeError:
                params[f'{field}_{ifo}'][params[f'tid_{ifo}'] >= 0] = sngls.get_column(f'{field}')
            except Exception as e:
                print(f"An error occurred: {e}")
                
    params['mean_time'] = [coinc.mean_if_greater_than_zero([params[f'time_{ifo}'][cnt] for ifo in ifos])[0]
                           for cnt in range(len(params['stat']))]
    params['mean_time'] = np.array(params['mean_time'])

    template_fields = ['mass1', 'mass2', 'spin1z', 'spin2z']
    for field in template_fields:
        params[f'{field}'] = bank[f'{field}'][:][bank_ids]
    
    return params

## Paths on CIT for first 5 O3 chunks:
 - /home/bhooshan.gadre/work/O3/sub_solar_search/chunk{2,4,5}_rerun_extrainj_rerun/output/
 - /home/bhooshan.gadre/work/O3/sub_solar_search/chunk1_rerun_extrainj_ssm2/output
 - /home/bhooshan.gadre/work/O3/sub_solar_search/chunk3_rerun_extrainj/output/

## Injection sets:
 - There are 4 injection sets per chunk: SSM1SEOBNRV4 and SSM{1,2,3}STT5
 - Each can be read separately using the code below

In [3]:
run_path = '/home/bhooshan.gadre/work/O3/sub_solar_search/chunk5_rerun_extrainj_rerun/output'

## Use the following to figure out chunk start (UTC) and chunk duration (sec) to be used later to access relevant files
!ls $run_path/bank/H1L1V1-BANK2HDF-*

## output should be H1L1V1-BANK2HDF-{chunk_start}-{chunk_duration}.hdf

/home/bhooshan.gadre/work/O3/sub_solar_search/chunk5_rerun_extrainj_rerun/output/bank/H1L1V1-BANK2HDF-1241010950-714094.hdf


In [4]:
chunk_start = 1241010950
chunk_duration = 714094

## Use the following to read different injection sets.

## To read injected parameters:

In [5]:
inj_type = 'SSM1SEOBNRV4' ## SSM{1,2,3}STT5

sfile =  HFile(os.path.join(run_path, 
                            f'{inj_type}_INJ_coinc/H1L1V1-HDFINJFIND_{inj_type}_INJ_INJECTIONS-{chunk_start}-{chunk_duration}.hdf'))
## The file handle below contain all the injection sets combined with missed-found info. 
## But the file cannot be used to extract trigger template parameters.

# allinj = HFile(os.path.join(run_path, 
                            # f'allinj/H1L1V1-HDFINJFIND_ALL_INJECTIONS-{chunk_start}-{chunk_duration}.hdf'))

In [6]:
sfile['injections'].keys()

<KeysViewHDF5 ['coa_phase', 'distance', 'eff_dist_h', 'eff_dist_l', 'eff_dist_v', 'end_time', 'inclination', 'latitude', 'longitude', 'mass1', 'mass2', 'optimal_snr_H1', 'optimal_snr_L1', 'optimal_snr_V1', 'polarization', 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z']>

In [7]:
found_ids = sfile['found_after_vetoes']['injection_index'][:]

found_injection = {k: sfile['injections'][k][:][found_ids] for k in sfile['injections'].keys()}
# found_injection

## To read found injection information
 - trigger template (same for all ifos due to exact coincidence search), stat, FAR, ifo specofic SNRs, phases, times


In [8]:
merge_trig_temp = os.path.join(run_path, 
                               f'{inj_type}_INJ_coinc/{{}}-HDF_TRIGGER_MERGE_{inj_type}_INJ_INJECTIONS-{chunk_start}-{chunk_duration}.hdf'
                              )

In [9]:
bank_file = os.path.join(run_path, 
                         f'bank/H1L1V1-BANK2HDF-{chunk_start}-{chunk_duration}.hdf')
bank = HFile(bank_file, 'r')

In [10]:
section = 'found_after_vetoes'
params_eob_inj = get_trigger_info(section, sfile, bank, merge_trig_temp)

# tid_{ifo} field is really irrelevant after fetching all the information. Those can/should be removed to save memory. 

reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [11]:
params_eob_inj.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

## Use the following to read full_data (real data foreground and background)

In [12]:
bank_file = os.path.join(run_path, 
                         f'bank/H1L1V1-BANK2HDF-{chunk_start}-{chunk_duration}.hdf') #es la misma que la de arriba 
merge_trig_temp = os.path.join(run_path, 
                               f'full_data/{{}}-HDF_TRIGGER_MERGE_FULL_DATA-{chunk_start}-{chunk_duration}.hdf')
statmap_file = os.path.join(run_path, 
                            f'full_data/H1L1V1-COMBINE_STATMAP_FULL_DATA-{chunk_start}-{chunk_duration}.hdf')

In [13]:
bank = HFile(bank_file, 'r')
sfile = HFile(statmap_file, 'r')

In [14]:
section = 'foreground' # 'background', 'background_exc'
paramsf = get_trigger_info(section, sfile, bank, merge_trig_temp)

reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [15]:
paramsf

# tid_{ifo} field is really irrelevant after fetching all the information. Those can/should be removed to save memory.

{'stat': array([-3.3924296 , -5.640567  , -5.3295374 , ...,  3.966397  ,
         1.3963292 , -0.74588513], dtype=float32),
 'ifar': array([6.88409937e-07, 6.56594854e-07, 6.58373804e-07, ...,
        1.55613850e-05, 2.56148508e-06, 1.01593787e-06]),
 'time_H1': array([-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
         1.24172479e+09,  1.24172482e+09,  1.24172483e+09]),
 'tid_H1': array([        -1,         -1,         -1, ...,  677756983, 1170487712,
        1787500513]),
 'coa_phase_H1': array([-1.        , -1.        , -1.        , ..., -0.10605115,
        -0.25043085, -0.62821639]),
 'snr_H1': array([-1.        , -1.        , -1.        , ...,  6.08852339,
         4.79349661,  5.447474  ]),
 'time_L1': array([ 1.24101397e+09,  1.24101399e+09,  1.24101400e+09, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]),
 'tid_L1': array([1673647446, 1171055633, 2406862606, ...,         -1,         -1,
                -1]),
 'coa_phase_L1': array([-2.94249439, -2

In [16]:
# section = 'background'
# paramsb = get_trigger_info(section, sfile, bank)